Using the MobileNet model and transfer learning to classify people as authorized or not authorized.

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense, GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np

In [ ]:
# load the base model
mobile_net = MobileNet()

In [ ]:
def prepare_image(path):
    img = image.load_img(path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [ ]:
# test an image
preprocessed_image = prepare_image("authorized.jpg")
predictions = mobile_net.predict(preprocessed_image)
print(predictions.shape)
results = imagenet_utils.decode_predictions(predictions)
print(results)

In [ ]:
# now to implement transfer learning on the model
base_model = MobileNet(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

In [ ]:
# create the model to train
model=Model(inputs=base_model.input, outputs=preds)

In [ ]:
# freeze the base model layers
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [ ]:
# create the data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
train_generator = train_datagen.flow_from_directory('data', target_size=(224,224), color_mode='rgb', batch_size=32, class_mode='categorical', shuffle=True)

In [ ]:
# compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train the model
step_size_train = train_generator.n//train_generator.batch_size
model.fit(x=train_generator, steps_per_epoch=step_size_train, epochs=2)


In [ ]:
# test the new model
img = image.load_img('data/authorized/authorized.jpg', target_size=(224, 224))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor = img_tensor/255

predictions = model.predict(img_tensor)
print(predictions)
# convert predictions to class labels
class_labels = train_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}
predictions = np.argmax(predictions, axis=1)
print(predictions)

In [ ]:
# save the model weights
model.save_weights('models/model_weights.h5')